# Exam Project | Face Recognizer
By **Pernille Lørup & Stephan Djurhuus**  
Institute **CPHBusiness**  

Education **Software Development**  
Elective **Artificial Intelligence**  

## Package Imports

In [ ]:
import cv2
import numpy as np

import tensorflow as tf
from tensorflow.keras import models, backend
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Dropout

## Module Imports

In [ ]:
import sys
sys.path.append('../')
from FaceDetector import FaceDetector
from FaceRecognizer import FaceRecognizer
from DataGenerator import DataGenerator
from Utilities import Displayer

## Global Constants

In [ ]:
classes = ['dad', 'mom', 'son', 'daughter']

dad_path = '../data/PersonGroup/Family1-Dad'
mom_path = '../data/PersonGroup/Family1-Mom'
son_path = '../data/PersonGroup/Family1-Son'
daughter_path = '../data/PersonGroup/Family1-Daughter'

## Data Generation

In [ ]:
(train_dad_data, test_dad_data) = DataGenerator.generate(dad_path, 150, label=0, test_size=.2)
(train_mom_data, test_mom_data) = DataGenerator.generate(mom_path, 150, label=1, test_size=.2)
(train_son_data, test_son_data) = DataGenerator.generate(son_path, 150, label=2, test_size=.2)
(train_daughter_data, test_daughter_data) = DataGenerator.generate(daughter_path, 150, label=3, test_size=.2)

(train_images, train_labels) = DataGenerator.merge_shuffle([train_dad_data, train_mom_data, train_son_data, train_daughter_data])
(test_images, test_labels) = DataGenerator.merge_shuffle([test_dad_data, test_mom_data, test_son_data, test_daughter_data])

# greyscalling
train_images_gray = np.array([cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) for image in train_images])
test_images_gray = np.array([cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) for image in test_images])

## Data Validations

In [ ]:
Displayer.images(train_images[200:], 20)
Displayer.images(train_images_gray[200:], 20)

## Model Configurations

### Convolutional Neural Network

In [ ]:
cnn_model = models.Sequential([
    Conv2D(25, (3, 3), activation='relu', input_shape=(100, 100, 3)),
    Dropout(.2),
    MaxPooling2D((2, 2)),
    Conv2D(50, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(50, (3, 3), activation='relu'),
    Flatten(),
    Dense(50, activation='relu'),
    Dense(len(classes))
])

### Artificial Neural Network

**Softmax** ≈ 50% accuracy  
**Sigmoid** ≈ 50% accuracy  
**TanH** ≈ 50% accuracy

In [ ]:
def positive_softmax(input_data):
    relu_data = backend.relu(input_data)        # excluding negative values
    softmax_data = backend.softmax(relu_data)   # 
    return softmax_data

ann_model = models.Sequential([
    Flatten(input_shape=(100, 100)),
    Dense(500, activation='relu'),
    Dense(250, activation='sigmoid'),
    Dense(len(classes)),
])

## Face Recognizer Instantiations

In [ ]:
cnn_recognizer = FaceRecognizer(classes, model=cnn_model)
ann_recognizer = FaceRecognizer(classes, model=ann_model)

## Model Training

In [ ]:
cnn_recognizer.train_model((train_images, train_labels), (test_images, test_labels), epochs=5)
# ann_recognizer.train_model((train_images_gray, train_labels), (test_images_gray, test_labels), epochs=15)

## Test Model

In [ ]:
# cnn_recognizer.test_model(test_image, test_labels)

In [ ]:
# ann_recognizer.test_model(test_image_gray, test_labels)

## Save & Load Model with History

In [ ]:
# cnn_recognizer.save_model('../models/cnn_model_3of4')
# cnn_recognizer.load_model('../models/cnn_model_1')

In [ ]:
# ann_recognizer.save_model('../models/ann_model_2of4')
ann_recognizer.load_model('../models/ann_model_3of4')

## Model Train History

In [ ]:
Displayer.acc_history(cnn_recognizer.history, 'CNN')
Displayer.acc_history(ann_recognizer.history, 'ANN')

## Model Confusion Matrix

### Convolutional Neural Network

In [ ]:
predictions = cnn_recognizer.recognize_many(test_images)
Displayer.conf_matrix(predictions, test_labels, 'CNN')

### Artificial Neural Network

In [ ]:
predictions = ann_recognizer.recognize_many(test_images_gray)
Displayer.conf_matrix(predictions, test_labels, 'ANN')

## Model Validations

In [ ]:
test_image_path = '../data/identification1.jpg'
test_image = cv2.imread(test_image_path)
test_image_gray = cv2.cvtColor(test_image, cv2.COLOR_BGR2GRAY)

faceDetector = FaceDetector('FACE_ALT2')
faces_data = faceDetector.face_details(test_image)

### Convolutional Neural Network

In [ ]:
cnn_prediction_data = cnn_recognizer.face_predictions(faces_data)
Displayer.mark_predictions(test_image.copy(), cnn_prediction_data, classes)

### Artificial Neural Network

In [ ]:
ann_prediction_data = ann_recognizer.face_predictions(faces_data, gray=True)
Displayer.mark_predictions(test_image.copy(), ann_prediction_data, classes)